# BlackBelt Consulting

## 1. Benson Project

### 1.1 Data Collection

In [1]:
import pandas as pd
import datetime

In [16]:
numdays = 21  # just setting the number of days we want to take back 1 week from 06-29
# we can increase it later, this is just so we can start with a "small" data set

In [17]:
date_time_str = '2019-06-22 08:15:27.243860'  # penultimate date of the range in the MTA website
date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')  
# transforming it to a date_time object
date_time_obj

datetime.datetime(2019, 6, 22, 8, 15, 27, 243860)

In [18]:
date_list = [date_time_obj - datetime.timedelta(days=x) for x in range(0, numdays, 7)]
# creating a list of dates that starts on the penultimate date and goes back the numdays we've set 
# (jumping 7 days each time)
date_list

[datetime.datetime(2019, 6, 22, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 15, 8, 15, 27, 243860),
 datetime.datetime(2019, 6, 8, 8, 15, 27, 243860)]

In [19]:
# transforming the dates into strings and putting in a list:

url_dates = []

for i in date_list:
    year = str(i.year).replace("20", "")
    day = str(i.day)
    if i.day < 10:
        day = "0" + day
    month = str(i.month)
    if i.month < 10:
        month = "0" + month
    date_str = year + month + day
    url_dates.append(date_str)

url_dates

['190622', '190615', '190608']

In [20]:
# starting the dataframe with the last available date, which is june 29, 19:
df = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_190629.txt")
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581


In [21]:
# concatenating each new date to the original dataframe:
for url in url_dates:
    df2 = pd.read_csv(
        "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt".format(url))
    df = pd.concat([df, df2], ignore_index=True)

In [23]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 823385 entries, 0 to 823384
Data columns (total 11 columns):
C/A                                                                     823385 non-null object
UNIT                                                                    823385 non-null object
SCP                                                                     823385 non-null object
STATION                                                                 823385 non-null object
LINENAME                                                                823385 non-null object
DIVISION                                                                823385 non-null object
DATE                                                                    823385 non-null object
TIME                                                                    823385 non-null object
DESC                                                                    823385 non-null object
ENTRIES                           

In [25]:
df.columns = [column.strip() for column in df.columns]

In [26]:
df["DATE_TIME"] = pd.to_datetime(df["DATE"] + " " + df["TIME"])

In [29]:
# Data from - Jun 2019
df.DATE.value_counts().sort_index()

06/01/2019    29066
06/02/2019    28968
06/03/2019    30022
06/04/2019    29513
06/05/2019    29484
06/06/2019    29213
06/07/2019    28745
06/08/2019    29030
06/09/2019    29133
06/10/2019    29351
06/11/2019    29113
06/12/2019    30023
06/13/2019    29276
06/14/2019    29008
06/15/2019    29605
06/16/2019    29322
06/17/2019    29421
06/18/2019    29599
06/19/2019    29614
06/20/2019    29556
06/21/2019    30360
06/22/2019    29217
06/23/2019    29170
06/24/2019    29479
06/25/2019    29437
06/26/2019    29417
06/27/2019    29464
06/28/2019    29779
Name: DATE, dtype: int64

In [33]:
df["TURNSTILE_ID"] = df["C/A"] + " " + df["UNIT"] + " " + df["SCP"] + " " + df["STATION"]

In [34]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,TURNSTILE_ID
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,00:00:00,REGULAR,7107725,2407457,2019-06-22 00:00:00,A002 R051 02-00-00 59 ST
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,04:00:00,REGULAR,7107738,2407465,2019-06-22 04:00:00,A002 R051 02-00-00 59 ST
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,08:00:00,REGULAR,7107761,2407491,2019-06-22 08:00:00,A002 R051 02-00-00 59 ST
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,12:00:00,REGULAR,7107858,2407541,2019-06-22 12:00:00,A002 R051 02-00-00 59 ST
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/22/2019,16:00:00,REGULAR,7108075,2407581,2019-06-22 16:00:00,A002 R051 02-00-00 59 ST


In [46]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(df
 .groupby(["TURNSTILE_ID", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(30)

,TURNSTILE_ID,DATE_TIME,ENTRIES
583293,R174 R034 00-00-03 125 ST,2019-06-08 00:00:00,2
194099,N045 R187 01-00-02 81 ST-MUSEUM,2019-06-01 17:00:00,2
193942,N045 R187 01-00-01 81 ST-MUSEUM,2019-06-02 17:00:00,2
194324,N045 R187 01-06-00 81 ST-MUSEUM,2019-06-11 21:00:00,2
660217,R249 R179 01-05-00 86 ST,2019-06-27 00:00:00,2
679477,R290 R161 00-00-00 KINGSBRIDGE RD,2019-06-07 05:00:00,2
660221,R249 R179 01-05-00 86 ST,2019-06-27 04:00:00,2
583125,R174 R034 00-00-02 125 ST,2019-06-08 00:00:00,2
193937,N045 R187 01-00-01 81 ST-MUSEUM,2019-06-01 17:00:00,2
194104,N045 R187 01-00-02 81 ST-MUSEUM,2019-06-02 17:00:00,2


In [47]:
# On some days, we seem to have two entries for same time.  Let's take a look
mask = ((df["TURNSTILE_ID"] == "R174 R034 00-00-03 125 ST") &
(df["DATE_TIME"].dt.date == datetime.datetime(2019, 6, 8).date()))

df[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,TURNSTILE_ID
558895,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,00:00:00,REGULAR,3828801,2853462,2019-06-08 00:00:00,R174 R034 00-00-03 125 ST
558896,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,00:00:00,RECOVR AUD,3828804,2853463,2019-06-08 00:00:00,R174 R034 00-00-03 125 ST
558897,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,08:00:00,REGULAR,3828931,2853610,2019-06-08 08:00:00,R174 R034 00-00-03 125 ST
558898,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,12:00:00,REGULAR,3829164,2853787,2019-06-08 12:00:00,R174 R034 00-00-03 125 ST
558899,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,16:00:00,REGULAR,3829514,2854060,2019-06-08 16:00:00,R174 R034 00-00-03 125 ST
558900,R174,R034,00-00-03,125 ST,1,IRT,06/08/2019,20:00:00,REGULAR,3829854,2854357,2019-06-08 20:00:00,R174 R034 00-00-03 125 ST


In [48]:
# On some days, we seem to have two entries for same time.  Let's take a look
mask = ((df["TURNSTILE_ID"] == "N045 R187 01-00-02 81 ST-MUSEUM") &
(df["DATE_TIME"].dt.date == datetime.datetime(2019, 6, 1).date()))

df[mask]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,TURNSTILE_ID
666700,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,01:00:00,REGULAR,4323386,989690,2019-06-01 01:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666701,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,05:00:00,REGULAR,4323388,989692,2019-06-01 05:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666702,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,09:00:00,REGULAR,4323394,989696,2019-06-01 09:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666703,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,13:00:00,REGULAR,4323510,989797,2019-06-01 13:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666704,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,17:00:00,REGULAR,4324002,989878,2019-06-01 17:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666705,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,17:00:00,RECOVR AUD,4323998,989878,2019-06-01 17:00:00,N045 R187 01-00-02 81 ST-MUSEUM
666706,N045,R187,01-00-02,81 ST-MUSEUM,BC,IND,06/01/2019,21:00:00,REGULAR,4324268,989897,2019-06-01 21:00:00,N045 R187 01-00-02 81 ST-MUSEUM


In [49]:
# Looking over more examples, these duplicates don't seem to show significant differences. 
# Therefore, we will get rid of the duplicate entries:
df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)

In [50]:
# No problems anymore:
(df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(15)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2019-06-01 00:00:00,1
548916,R139,R031,04-00-01,34 ST-PENN STA,2019-06-28 12:00:00,1
548906,R139,R031,04-00-01,34 ST-PENN STA,2019-06-26 20:00:00,1
548907,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 00:00:00,1
548908,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 04:00:00,1
548909,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 08:00:00,1
548910,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 12:00:00,1
548911,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 16:00:00,1
548912,R139,R031,04-00-01,34 ST-PENN STA,2019-06-27 20:00:00,1
548913,R139,R031,04-00-01,34 ST-PENN STA,2019-06-28 00:00:00,1


For our data we are focusing only in the entries since we believe this is a better approach to gauge the number of people focusing on the ads.

In [51]:
# Drop Exits and Desc Column.  To prevent errors in multiple run of cell, errors on drop is ignored
df = df.drop(["EXITS", "DESC"], axis=1, errors="ignore")

In [68]:
df.reset_index(drop=True)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,DATE_TIME,TURNSTILE_ID
0,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,21:00:00,5554,2019-06-28 21:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
1,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,17:00:00,5554,2019-06-28 17:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
2,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,13:00:00,5554,2019-06-28 13:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
3,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,09:00:00,5554,2019-06-28 09:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
4,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,05:00:00,5554,2019-06-28 05:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
5,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,01:00:00,5554,2019-06-28 01:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
6,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/27/2019,21:00:00,5554,2019-06-27 21:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
7,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/27/2019,17:00:00,5554,2019-06-27 17:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
8,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/27/2019,13:00:00,5554,2019-06-27 13:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT
9,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/27/2019,09:00:00,5554,2019-06-27 09:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT


In [70]:
# There seems to be a problem with 23rd street (and others) because different stations are named the same way:
df[df["STATION"] == "23 ST"].groupby(["LINENAME"]).count()

,C/A,UNIT,SCP,STATION,DIVISION,DATE,TIME,ENTRIES,DATE_TIME,TURNSTILE_ID
LINENAME,,,,,,,,,,
1,1184,1184,1184,1184,1184,1184,1184,1184,1184,1184
6,4277,4277,4277,4277,4277,4277,4277,4277,4277,4277
CE,1878,1878,1878,1878,1878,1878,1878,1878,1878,1878
FM,1724,1724,1724,1724,1724,1724,1724,1724,1724,1724
NRW,1769,1769,1769,1769,1769,1769,1769,1769,1769,1769


In [71]:
# Let's create a new column that concatenates the station and the lines to differenciate them:
df["STATION_LINE"] = df["STATION"] + " " + df["LINENAME"]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,DATE_TIME,TURNSTILE_ID,STATION_LINE
205962,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,21:00:00,5554,2019-06-28 21:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT,RIT-ROOSEVELT R
205961,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,17:00:00,5554,2019-06-28 17:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT,RIT-ROOSEVELT R
205960,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,13:00:00,5554,2019-06-28 13:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT,RIT-ROOSEVELT R
205959,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,09:00:00,5554,2019-06-28 09:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT,RIT-ROOSEVELT R
205958,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/28/2019,05:00:00,5554,2019-06-28 05:00:00,TRAM2 R469 00-05-01 RIT-ROOSEVELT,RIT-ROOSEVELT R


In [80]:
unique_stations = df["STATION_LINE"].unique()
sorted(unique_stations)

['1 AV L',
 '103 ST 1',
 '103 ST 6',
 '103 ST BC',
 '103 ST-CORONA 7',
 '104 ST A',
 '104 ST JZ',
 '110 ST 6',
 '111 ST 7',
 '111 ST A',
 '111 ST J',
 '116 ST 23',
 '116 ST 6',
 '116 ST BC',
 '116 ST-COLUMBIA 1',
 '121 ST JZ',
 '125 ST 1',
 '125 ST 23',
 '125 ST 456',
 '125 ST ACBD',
 '135 ST 23',
 '135 ST BC',
 '137 ST CITY COL 1',
 '138/GRAND CONC 45',
 '14 ST 123FLM',
 '14 ST ACEL',
 '14 ST FLM123',
 '14 ST-UNION SQ 456LNQRW',
 '14 ST-UNION SQ LNQR456W',
 '145 ST 1',
 '145 ST 3',
 '145 ST ABCD',
 '149/GRAND CONC 245',
 '14TH STREET 1',
 '15 ST-PROSPECT FG',
 '155 ST BD',
 '155 ST C',
 '157 ST 1',
 '161/YANKEE STAD 4BD',
 '161/YANKEE STAD BD4',
 '163 ST-AMSTERDM C',
 '167 ST 4',
 '167 ST BD',
 '168 ST 1AC',
 '168 ST AC1',
 '169 ST F',
 '170 ST 4',
 '170 ST BD',
 '174 ST 25',
 '174-175 STS BD',
 '175 ST A',
 '176 ST 4',
 '18 AV D',
 '18 AV F',
 '18 AV N',
 '18 ST 1',
 '181 ST 1',
 '181 ST A',
 '182-183 STS BD',
 '183 ST 4',
 '190 ST A',
 '191 ST 1',
 '2 AV F',
 '20 AV D',
 '20 AV N',
